# Set up

In [ ]:
# Install packages

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score

In [ ]:
# Setting the Seed (pandas, statsmodels, matplotlib and y_data_profiling rely on 
# numpy's random generator, and thus, we need to set the seed in numpy)
seed = 2024
np.random.seed(seed)

In [ ]:
# Importing the data 
diet_df = pd.read_csv("diet.csv")
recipes_df = pd.read_csv("recipes.csv")
requests_df = pd.read_csv("requests.csv")
reviews_df = pd.read_csv("reviews.csv")

## Cleaning diet.csv

In [ ]:
# Dataset is already clean, only one missing value -> replace with most common diet

# Clean Diet
diet_cleaned = diet_df 

# Replace the missing values with the most common diet
diet_cleaned.loc[diet_cleaned["AuthorId"] == "646062A", "Diet"] = "Vegetarian"

# Dummy Encode the diet column to int64
diet_cleaned = pd.get_dummies(diet_cleaned, columns=["Diet"], prefix=["Diet"])

In [ ]:
# cut the Age column into 4 eqaully sized bins (18-33), (34-48), (49-63), (64-79)
diet_cleaned["Age_Group"] = pd.cut(diet_cleaned["Age"], bins=4, labels=["Young_Adults", "Middle_Adults", "Old_Adults", "Seniors"])


In [ ]:
# dummy encode the age groups
diet_cleaned = pd.get_dummies(diet_cleaned, columns=["Age_Group"])

## Cleaning requests.csv

In [ ]:
requests_cleaned = requests_df.copy()

In [ ]:
# Replace negative Time values with 0 (other option: Really high value)
requests_cleaned.loc[requests_cleaned["Time"] < 0, "Time"] = 0

# convert the Time column to int64
requests_cleaned["Time"] = requests_cleaned["Time"].astype("int")

In [ ]:
# make column HighCalories integer with no decimal places
requests_cleaned["HighCalories"] = requests_cleaned["HighCalories"].astype("int")

In [ ]:
# Handling the HighProtein column
requests_cleaned['HighProtein'] = requests_df['HighProtein'].map({'Yes': 1, 'Indifferent': 0})

# Handling the LowSugar column
requests_cleaned['LowSugar'] = requests_df['LowSugar'].map({'0': 1, 'Indifferent': 0})

In [ ]:
# Make column for healty requests if HighProtein is 1, LowFat is 1 and HighFiber is 1
requests_cleaned['Healthy'] = np.where((requests_cleaned['HighCalories'] == 0) & (requests_cleaned['HighProtein'] == 1) & (requests_cleaned['LowFat'] == 1) & (requests_cleaned['HighFiber'] == 1), 1, 0)

## Cleaning reviews.csv

In [ ]:
reviews_cleaned = reviews_df.copy()

In [ ]:
# Handling the Like Column 
reviews_cleaned['Like'] = reviews_cleaned['Like'].map({True: 1, False: 0})

## Cleaning recipes.csv

In [ ]:
recipes_cleaned = recipes_df.copy()

In [ ]:
# Creating extra categories doesnt give value
# Handling the RecipeCategory column
recipes_cleaned = pd.get_dummies(recipes_cleaned, columns=['RecipeCategory'])

In [ ]:
# calculate total time as the sum of prep time and cook time
recipes_cleaned["TotalTime"] = recipes_cleaned["PrepTime"] + recipes_cleaned["CookTime"]

In [ ]:
# Look for specific ingredients in the RecipeIngredientParts column
recipes_cleaned['Meat'] = recipes_cleaned['RecipeIngredientParts'].str.contains('chicken|beef|pork|lamb|turkey|duck|goose|fish|seafood|salmon|shrimp|crab|lobster|ham|bacon|sausage|meat|steak|veal|venison|bison|liver|lamb|poultry|meat|tuna|mackerel|trout|haddock|cod|catfish|tilapia|sardine|anchovy|halibut|swordfish|monkfish|eel|octopus|squid|clams|mussels|oysters|snail|game|partridge|pheasant|quail|rabbit|hare|wild boar|elk|moose|reindeer|buffalo|ostrich|emu|kangaroo|alligator|frog legs|snapper|grouper|barracuda|carp|pike|perch|flounder|sole|mahi mahi|red snapper|swordfish|branzino|char|chub|mullet|ray|skate|sturgeon|turbot|whiting|arctic char|caviar|roe|scallop|sea urchin|abalone|crayfish|langoustine|prawn|cuttlefish|jellyfish|stingray|barramundi|black cod|bluefish|bonito|bream|burbot|butterfish|capelin|chimaera|conger|coral trout|dab|dace|dogfish|dorado|dory|drum|escolar|garfish|gilt-head bream|hake|herring|John Dory|kingfish|lamprey|lingcod|lumpsucker|marlin|milkfish|nile perch|northern pike|orange roughy|pacific saury|patagonian toothfish|pollock|pomfret|pompano|rockfish|rudd|sablefish|sanddab|sculpin|sea bass|shark|sheepshead|silver carp|skipjack tuna|smelt|snook|sockeye salmon|sole|sprat|striped bass|sturgeon|surimi|tarpon|tench|wahoo|walleye|warmouth|weakfish|whitebait|whitefish|yellowtail|zebrafish', case=False)
recipes_cleaned['Meat'] = recipes_cleaned['Meat'].astype(int)

recipes_cleaned['Vegetables'] = recipes_cleaned['RecipeIngredientParts'].str.contains('carrot|potato|tomato|onion|garlic|broccoli|spinach|cucumber|lettuce|celery|cabbage|cauliflower|corn|pepper|peas|beans|asparagus|eggplant|zucchini|squash|pumpkin|radish|beet|turnip|sprouts|vegetable|vegetables|kale|bell pepper|sweet potato|cherry tomato|artichoke|bok choy|brussels sprouts|butternut squash|leek|okra|snow peas|snap peas|green beans|fennel|watercress|arugula|chard|collard greens|endive|escarole|kohlrabi|mustard greens|pattypan squash|romanesco broccoli|rutabaga|sugar snap peas|swiss chard|tatsoi|water chestnut|yam|daikon|lotus root|bamboo shoots|jicama|plantain|radicchio|turnip greens|water spinach|bitter melon|chayote|chicory|dandelion greens|edamame|lotus root|nopales|portobello mushroom|shiitake mushroom|maitake mushroom|enoki mushroom|oyster mushroom|mushroom|zucchini blossom|artichoke|sunchoke|beets|cardoon|salsify|kale|endive|acorn squash|alfalfa sprout|amaranth|anise|arracacha|arrowroot|aubergine|avocado|azuki bean|banana squash|barbarea verna|basil|bean sprout|beet greens|belgian endive|bell pepper|black bean|black-eyed pea|borage|broad bean|broccoflower|broccolini|brussels sprout|butter lettuce|butternut lettuce|cactus|calabash|camas root|canna|cape gooseberry|caper|cardinal bean|cassava|catsear|cauliflower|cayenne pepper|celeriac|celtuce|chayote|cherry pepper|chickpea|chicory|chili pepper|chinese artichoke|chinese broccoli|chinese cabbage|chinese mallow|chive|cilantro|collard|common bean|courgette|courgette flower|cowpea|cress|crookneck squash|cucumber|daikon|delicata squash|dill|drumstick|earthnut pea|elephant garlic|endive|epazote|fava bean|fennel|fenugreek|fluted pumpkin|french bean|frisee|garbanzo|gem squash|ginger|golden samphire|good king henry|grape leaves|green bean|green soybean|guar|habanero|haricot bean|horseradish|hubbard squash|hyacinth bean|iceberg lettuce|jerusalem artichoke|jicama|jute|kai-lan|kidney bean|kohlrabi|komatsuna|kombu|kuka|lacinto|lagos bologi|land cress|laver|leek|lemon grass|lentil|lettuce|lima bean|linguica|lo bok|lotus root|luffa|mache|malabar spinach|mangetout|mizuna greens|molokhia|monstera|morel|morogo|mung bean|mustard|napa cabbage|new zealand spinach|nopale|oceanic whitetip shark|okra|onion|oregano|orache|pak choy|paprika|parsley|parsnip|pea|peanut|pearl onion|pigeon pea|pignut|pimento|pinto bean|pisello|plantain|poblano|pokeweed|potato|prairie turnip|pumpkin|purslane|radicchio|radish|rape|rapini|raspberry|red bean|red cabbage|red pepper|rhubarb|ricebean|rocket|romanescu|romano bean|rosemary|runner bean|rutabaga|rye|saffron|sage|salad savoy|salsify|samphire|scallion|scarlet runner bean|sea kale|seakale beet|sesame|shallot|shiitake|skirret|sloe|snake bean|snow pea|sorrel|sour cherry|southern pea|soybean|spaghetti squash|spinach|spring onion|squash|squash blossom|swede|sweet corn|sweet pepper|sweet potato|swiss chard|taro|tatsoi|thyme|tomatillo|tomato|topinambour|tubers|turmeric|turnip|turnip greens|ugli fruit|ursolic acid|uva-ursi|vates|water chestnut|water spinach|watercress|wax bean|wax gourd|welsh onion|west indian gherkin|white bean|white radish|wild leek|wild rice|winged bean|winter melon|winter squash|yam bean|yardlong bean|yellow squash|yu choy|zucchini', case=False)
recipes_cleaned['Vegetables'] = recipes_cleaned['Vegetables'].astype(int)

recipes_cleaned['Dairy'] = recipes_cleaned['RecipeIngredientParts'].str.contains('milk|cheese|yogurt|cream|butter|buttermilk|sour cream|cottage cheese|mascarpone|ricotta|cream cheese|mozzarella|cheddar|brie|feta|parmesan|gouda|havarti|blue cheese|goat cheese|swiss cheese|provolone|muenster cheese|asiago|colby jack|mozzarella sticks|queso fresco|queso blanco|halloumi|neufchâtel|ghee|clotted cream', case=False)
recipes_cleaned['Dairy'] = recipes_cleaned['Dairy'].astype(int)

# specific things people dont like
recipes_cleaned['Olives'] = recipes_cleaned['RecipeIngredientParts'].str.contains('olives', case=False) 
recipes_cleaned['Olives'] = recipes_cleaned['Olives'].astype(int)

recipes_cleaned['Mushroom'] = recipes_cleaned['RecipeIngredientParts'].str.contains('mushrooms|mushroom|fungus|champignon|champignons|oyster|morel', case=False)
recipes_cleaned['Mushroom'] = recipes_cleaned['Mushroom'].astype(int)

recipes_cleaned['Asparagus'] = recipes_cleaned['RecipeIngredientParts'].str.contains('asparagus', case=False)
recipes_cleaned['Asparagus'] = recipes_cleaned['Asparagus'].astype(int)

recipes_cleaned['Garlic'] = recipes_cleaned['RecipeIngredientParts'].str.contains('garlic', case=False)
recipes_cleaned['Garlic'] = recipes_cleaned['Garlic'].astype(int)

In [ ]:
# Add Number of Quantities
recipes_cleaned['Number of Ingredients'] = recipes_cleaned['RecipeIngredientParts'].str.count(',') + 1

In [ ]:
# Drop unnecessary columns for now
recipes_cleaned.drop(columns=['Name', "RecipeIngredientParts", "RecipeIngredientQuantities", 'CookTime', 'PrepTime', 'RecipeServings', 'RecipeYield', 'SodiumContent', 'SaturatedFatContent'], inplace=True)

## Merge all datasets

In [ ]:
full_df = pd.merge(requests_cleaned, reviews_cleaned, on=['RecipeId', 'AuthorId'])
full_df = pd.merge(full_df, recipes_cleaned, on=['RecipeId'])
full_df = pd.merge(full_df, diet_cleaned, on=['AuthorId'])

## Cross feature engineering

In [ ]:
# Engineer complexity
full_df = full_df.assign(complex_to_cook = np.where((full_df['TotalTime'] > 1800) & (full_df['Number of Ingredients'] >= 15), 1, 0))

full_df = full_df.assign(easy_to_cook = np.where((full_df['TotalTime'] <= 900) & (full_df['Number of Ingredients'] <= 7), 1, 0))

# make column medium_to_cook if not complex and not easy
full_df = full_df.assign(medium_to_cook = np.where((full_df['complex_to_cook'] == 0) & (full_df['easy_to_cook'] == 0), 1, 0))

In [ ]:
# HighFiberRequestFulfilled -> 1, if FiberContent is more than 10
full_df = full_df.assign(HighFiberRequestFulfilled = np.where((full_df['HighFiber'] == 1) & (full_df['FiberContent'] >= 2.5), 1, 0))
full_df = full_df.assign(HighFiberRequestFulfilled = np.where((full_df['HighFiber'] == 0) & (full_df['FiberContent'] < 2.5), 1, 0))

In [ ]:
# LowFatRequestFulfilled -> 1, if LowFat is 1, and FatContent is less than 3
full_df = full_df.assign(LowFatRequestFulfilled = np.where((full_df['LowFat'] == 1) & (full_df['FatContent'] < 13), 1, 0))
full_df = full_df.assign(LowFatRequestFulfilled = np.where((full_df['LowFat'] == 0) & (full_df['FatContent'] >= 13), 1, 0))

In [ ]:
# unhealthy nutrients
full_df["total_unhealthy_grams"] = full_df["FatContent"] + full_df["CarbohydrateContent"]

In [ ]:
recipes_cleaned["TotalGrams"] = recipes_cleaned["FatContent"] + recipes_cleaned["ProteinContent"] + recipes_cleaned["CarbohydrateContent"]

In [ ]:
full_df["TimeDifference"] = full_df["Time"] - full_df["TotalTime"]

In [ ]:
# make new feature based on Age and FatContent
full_df = full_df.assign(YoungHighProtein = np.where((full_df['Age_Group_Young_Adults'] == 1) & (full_df['FatContent'] >= 13), 1, 0))
full_df = full_df.assign(MiddleHighProtein = np.where((full_df['Age_Group_Middle_Adults'] == 1) & (full_df['FatContent'] >= 13), 1, 0))
full_df = full_df.assign(OldHighProtein = np.where((full_df['Age_Group_Old_Adults'] == 1) & (full_df['FatContent'] >= 13), 1, 0))
full_df = full_df.assign(SeniorsHighProtein = np.where((full_df['Age_Group_Seniors'] == 1) & (full_df['FatContent'] >= 13), 1, 0))

## Splitting dataset

In [ ]:
full_df_submission = full_df[full_df["Like"].isnull()]
full_df_modelling = full_df[full_df["Like"].notnull()]

In [ ]:
full_df_modelling = full_df_modelling[full_df_modelling['TotalTime'] < 10000]

## Create Train and Test 

In [ ]:
from sklearn.model_selection import train_test_split


X = full_df_modelling.drop(["AuthorId", "RecipeId", "Like", "TestSetId", "Time", "TotalTime", "Rating"], axis=1)
y = full_df_modelling["Like"]

# X_train, X_test, y_train, y_test = \
#   train_test_split(X, y,
#                    test_size=0.3, 
#                    shuffle=True,
#                    random_state=seed)

## GradientBooster

Runs approximately 17 minutes on Mac 2,6 GHz 6-Core Intel Core i7 with 16GB RAM

In [ ]:
# Create the model
gb_model = GradientBoostingClassifier(n_estimators=4000)

# Fit the model
gb_model.fit(X, y)

#predictions = gb_model.predict(X_test)

# Calculate the balanced accuracy score
#print(balanced_accuracy_score(y_test, predictions))

In [ ]:
# Submission
submission_predictions = gb_model.predict(full_df_submission.drop(["AuthorId", "RecipeId", "Like", "TestSetId", "Time", "TotalTime", "Rating"], axis=1))

output = pd.DataFrame({'id': full_df_submission.TestSetId.astype(int), 'prediction': submission_predictions.astype(int)})
output = output.sort_values('id')
output.to_csv('../predictions_analytics_acrobats_6.csv', index=False)